In [1]:
import os
import numpy as np
import torch
import pickle

from torch.utils.data import DataLoader
from generate_data import generate_vrp_data
from utils import load_model
from problems import CVRP

In [2]:
# %matplotlib inline
# from matplotlib import pyplot as plt

# from matplotlib.collections import PatchCollection
# from matplotlib.patches import Rectangle
# from matplotlib.lines import Line2D

# # Code inspired by Google OR Tools plot:
# # https://github.com/google/or-tools/blob/fb12c5ded7423d524fc6c95656a9bdc290a81d4d/examples/python/cvrptw_plot.py

# def discrete_cmap(N, base_cmap=None):
#   """
#     Create an N-bin discrete colormap from the specified input map
#     """
#   # Note that if base_cmap is a string or None, you can simply do
#   #    return plt.cm.get_cmap(base_cmap, N)
#   # The following works for string, None, or a colormap instance:

#   base = plt.cm.get_cmap(base_cmap)
#   color_list = base(np.linspace(0, 1, N))
#   cmap_name = base.name + str(N)
#   return base.from_list(cmap_name, color_list, N)

# def plot_vehicle_routes(data, route, ax1, markersize=5, visualize_demands=False, demand_scale=1, round_demand=False):
#     """
#     Plot the vehicle routes on matplotlib axis ax1.
#     """
    
#     # route is one sequence, separating different routes with 0 (depot)
#     routes = [r[r!=0] for r in np.split(route.cpu().numpy(), np.where(route==0)[0]) if (r != 0).any()]
#     depot = data['depot'].cpu().numpy()
#     locs = data['loc'].cpu().numpy()
#     demands = data['demand'].cpu().numpy() * demand_scale
#     capacity = demand_scale # Capacity is always 1
    
#     x_dep, y_dep = depot
#     ax1.plot(x_dep, y_dep, 'sk', markersize=markersize*4)
#     ax1.set_xlim(0, 1)
#     ax1.set_ylim(0, 1)
    
#     legend = ax1.legend(loc='upper center')
    
#     cmap = discrete_cmap(len(routes) + 2, 'nipy_spectral')
#     dem_rects = []
#     used_rects = []
#     cap_rects = []
#     qvs = []
#     total_dist = 0
#     for veh_number, r in enumerate(routes):
#         color = cmap(len(routes) - veh_number) # Invert to have in rainbow order
        
#         route_demands = demands[r - 1]
#         coords = locs[r - 1, :]
#         xs, ys = coords.transpose()

#         total_route_demand = sum(route_demands)
#         assert total_route_demand <= capacity
#         if not visualize_demands:
#             ax1.plot(xs, ys, 'o', mfc=color, markersize=markersize, markeredgewidth=0.0)
        
#         dist = 0
#         x_prev, y_prev = x_dep, y_dep
#         cum_demand = 0
#         for (x, y), d in zip(coords, route_demands):
#             dist += np.sqrt((x - x_prev) ** 2 + (y - y_prev) ** 2)
            
#             cap_rects.append(Rectangle((x, y), 0.01, 0.1))
#             used_rects.append(Rectangle((x, y), 0.01, 0.1 * total_route_demand / capacity))
#             dem_rects.append(Rectangle((x, y + 0.1 * cum_demand / capacity), 0.01, 0.1 * d / capacity))
            
#             x_prev, y_prev = x, y
#             cum_demand += d
            
#         dist += np.sqrt((x_dep - x_prev) ** 2 + (y_dep - y_prev) ** 2)
#         total_dist += dist
#         qv = ax1.quiver(
#             xs[:-1],
#             ys[:-1],
#             xs[1:] - xs[:-1],
#             ys[1:] - ys[:-1],
#             scale_units='xy',
#             angles='xy',
#             scale=1,
#             color=color,
#             label='R{}, # {}, c {} / {}, d {:.2f}'.format(
#                 veh_number, 
#                 len(r), 
#                 int(total_route_demand) if round_demand else total_route_demand, 
#                 int(capacity) if round_demand else capacity,
#                 dist
#             )
#         )
        
#         qvs.append(qv)
        
#     ax1.set_title('{} routes, total distance {:.2f}'.format(len(routes), total_dist))
#     ax1.legend(handles=qvs)
    
#     pc_cap = PatchCollection(cap_rects, facecolor='whitesmoke', alpha=1.0, edgecolor='lightgray')
#     pc_used = PatchCollection(used_rects, facecolor='lightgray', alpha=1.0, edgecolor='lightgray')
#     pc_dem = PatchCollection(dem_rects, facecolor='black', alpha=1.0, edgecolor='black')
    
#     if visualize_demands:
#         ax1.add_collection(pc_cap)
#         ax1.add_collection(pc_used)
#         ax1.add_collection(pc_dem)

In [3]:
# model, _ = load_model('pretrained/cvrp_100/')
# model, _ = load_model('workstation_output/cvrp_50/20211106/epoch-99.pt')
model, _ = load_model('workstation_output/cvrp_50/20220217/epoch-21.pt')
torch.manual_seed(1234)
SIZE=100
num_samples=100
# dataset = CVRP.make_dataset(size=100, num_samples=1000)
dataset = CVRP.make_dataset(size=SIZE, num_samples=num_samples)

  [*] Loading model from workstation_output/cvrp_50/20220217/epoch-21.pt


In [4]:
# for i in range(100):
#     data_dice = {}
#     data_dice['X'] = [float(ele[0]) for ele in dataset[i]['loc']]
#     data_dice['X'] = [float(dataset[i]['depot'][0])] + data_dice['X'] ## add depot on top
    
#     data_dice['Y'] = [float(ele[1]) for ele in dataset[i]['loc']]
#     data_dice['Y'] = [float(dataset[i]['depot'][1])] + data_dice['Y'] ## add depot on top
    
#     data_dice['Demand'] = [0.0] + [float(ele) for ele in dataset[i]['demand']]
#     data_dice['start_time'] = [int(ele[0]) for ele in dataset[i]['time_window']]
#     data_dice['end_time'] = [int(ele[1]) for ele in dataset[i]['time_window']]
#     data_dice['service_time'] = [0.0 for ele in range(len(dataset[i]['time_window']))]
#     import pandas as pd
#     df = pd.DataFrame(data_dice)
#     df['Demand'] = df['Demand'] * 100
#     df.to_csv("~/fast-routing/benchmark/gurobi/data/node50/ex"+str(i)+".csv", index=False)

In [6]:
# Need a dataloader to batch instances
dataloader = DataLoader(dataset, batch_size=1000)

# Make var works for dicts
batch = next(iter(dataloader))

# Run the model
model.eval()
model.set_decode_type('greedy')
with torch.no_grad():
    length, log_p, pi = model(batch, return_pi=True)
tours = pi

# Plot the results
# for i, (data, tour) in enumerate(zip(dataset, tours)):
#     fig, ax = plt.subplots(figsize=(10, 10))
#     plot_vehicle_routes(data, tour, ax, visualize_demands=False, demand_scale=50, round_demand=True)
#     fig.savefig(os.path.join('images', 'cvrp_{}.png'.format(i)))

In [19]:
##CREATE DATA FOR GUROBI##############################################    
# for i in range(num_samples):
#     data_dice = {}
#     data_dice['X'] = [float(ele[0]) for ele in model.state.coords[i]]
#     data_dice['Y'] = [float(ele[1]) for ele in model.state.coords[i]]
    
#     data_dice['Demand'] = [0.0] + [float(ele) for ele in model.state.demand[i]]
#     data_dice['start_time'] = [int(ele[0]) for ele in model.state.time_window[i]]
#     data_dice['end_time'] = [int(ele[1]) for ele in model.state.time_window[i]]
#     data_dice['service_time'] = [0.0 for ele in range(len(model.state.time_window[i]))]
#     import pandas as pd
#     df = pd.DataFrame(data_dice)
#     df['Demand'] = df['Demand'] * 100
#     df.to_csv("~/fast-routing/benchmark/gurobi/data/node" + str(SIZE) +"/ex"+str(i)+".csv", index=False)


# def make_data_model(model, index):
#     data_model = {}
#     data_model['time_matrix'] = []
#     size = len(model.state.coords[index])
#     coords = np.array(model.state.coords[index])
#     for i in range(size):
#         temp = []
#         for j in range(size):
#             temp.append(round(np.linalg.norm(coords[i]-coords[j]) * 1000))
#         data_model['time_matrix'].append(temp)

#     data_model['time_windows'] = []
#     time_window = model.state.time_window[index]
#     for i in range(size):
#         data_model['time_windows'].append((int(time_window[i][0])*1000, int(time_window[i][1])*1000))

#     data_model['demands'] = [0] + [round(float(demand)*1000) for demand in model.state.demand[index]]
#     data_model['num_vehicles'] = 100
#     data_model['vehicle_capacities'] = [1000] * data_model['num_vehicles']
#     data_model['depot'] = 0
#     return data_model

# for i in range(num_samples):
#     data = make_data_model(model, i)
#     with open("/home/asaki/fast-routing/benchmark/ortools/data/node" + str(SIZE) +"/ex"+str(i)+".pickle", mode="wb") as f:
#         pickle.dump(data, f)
#     with open("/home/asaki/fast-routing/benchmark/ortools_guided/data/node" + str(SIZE) +"/ex"+str(i)+".pickle", mode="wb") as f:
#         pickle.dump(data, f)

In [6]:
def to_depot_distance(coords, location, i):
    return np.linalg.norm(np.array(coords[location[i-1]]) - np.array(coords[location[i]]))

In [7]:
total_list = []
for instance_index in range(100):
    print(instance_index)
    total = 0
    for i in range(len(model.state.current_time_list)):
        if instance_index == 0 and True:
            print(model.state.current_time_list[i][instance_index], dataset.data[instance_index]['time_window'][pi[instance_index][i]])
#             print((model.state.coords[instance_index][pi[instance_index][i]] - model.state.coords[instance_index][pi[instance_index][i+1]]).norm(p=2, dim=-1))
        current = float(model.state.current_time_list[i][instance_index])
        if i != 0 and current == 0.0:
#             print(to_depot_distance(model.state.coords[instance_index], tours[instance_index], i))
            total += previous + to_depot_distance(model.state.coords[instance_index], tours[instance_index], i)
        previous = current
    total_list.append(total)

0
tensor([6.]) tensor([ 6., 15.])
tensor([7.]) tensor([ 7., 11.])
tensor([7.5655]) tensor([6., 9.])
tensor([8.1021]) tensor([ 6., 13.])
tensor([8.8086]) tensor([ 8., 10.])
tensor([9.6028]) tensor([ 9., 14.])
tensor([10.0549]) tensor([ 8., 15.])
tensor([11.]) tensor([11., 13.])
tensor([11.4782]) tensor([11., 18.])
tensor([12.4364]) tensor([11., 13.])
tensor([12.7052]) tensor([ 2., 13.])
tensor([13.8948]) tensor([ 6., 15.])
tensor([14.3478]) tensor([ 6., 15.])
tensor([0.]) tensor([0., 0.])
tensor([5.]) tensor([ 5., 12.])
tensor([6.2200]) tensor([ 6., 10.])
tensor([7.]) tensor([ 7., 15.])
tensor([7.4560]) tensor([ 7., 13.])
tensor([8.6650]) tensor([7., 9.])
tensor([8.8653]) tensor([ 7., 14.])
tensor([9.2413]) tensor([ 7., 15.])
tensor([9.9031]) tensor([ 9., 16.])
tensor([10.4970]) tensor([ 9., 12.])
tensor([11.]) tensor([11., 14.])
tensor([11.4997]) tensor([11., 14.])
tensor([11.7620]) tensor([ 1., 12.])
tensor([0.]) tensor([0., 0.])
tensor([4.]) tensor([4., 6.])
tensor([5.6190]) tensor([

In [8]:
# for i, (data, tour) in enumerate(zip(dataset, tours)):
#     fig, ax = plt.subplots(figsize=(10, 10))
#     plot_vehicle_routes(data, tour, ax, visualize_demands=False, demand_scale=2010, round_demand=True)
#     # fig.savefig(os.path.join('images', 'cvrp_{}.png'.format(i)))

In [9]:
with open("result_"+str(SIZE)+".pickle", mode="wb") as f:
    pickle.dump(total_list, f)
total_list

[82.99759825319052,
 82.38348956406116,
 94.48665823414922,
 79.67075563967228,
 82.65153325349092,
 74.79701050370932,
 92.6741316318512,
 99.7903568521142,
 84.17076298594475,
 93.72370409034193,
 86.61766066402197,
 102.23833871260285,
 98.47720646113157,
 83.94603114016354,
 96.84458939731121,
 94.59226081520319,
 93.72196159884334,
 87.91991482675076,
 95.04481012560427,
 90.01011735200882,
 83.24563530087471,
 90.33420834317803,
 83.91088058799505,
 95.70737715810537,
 106.54280174151063,
 81.94437934085727,
 83.6538256034255,
 88.85481265187263,
 94.08097421750426,
 92.4094816558063,
 94.42349640093744,
 83.10252152383327,
 102.75083264708519,
 84.38056410104036,
 85.35255841910839,
 91.7836212515831,
 80.94397898763418,
 83.68261213973165,
 99.62037638574839,
 90.60206035897136,
 94.30689989775419,
 86.80764125660062,
 78.27575567364693,
 93.59226834774017,
 81.02405648306012,
 93.99700218439102,
 92.87009055912495,
 91.82641339302063,
 83.70967075973749,
 78.98581595905125,
 9

In [10]:
sum(total_list)

8803.19210061524

In [15]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def make_data_model(model, index):
    global data_model
    data_model = {}
    data_model['time_matrix'] = []
    size = len(model.state.coords[index])
    coords = np.array(model.state.coords[index])
    for i in range(size):
        temp = []
        for j in range(size):
            temp.append(round(np.linalg.norm(coords[i]-coords[j]) * 1000))
        data_model['time_matrix'].append(temp)

    data_model['time_windows'] = []
    time_window = model.state.time_window[index]
    for i in range(size):
        data_model['time_windows'].append((int(time_window[i][0])*1000, int(time_window[i][1])*1000))

    data_model['demands'] = [0] + [round(float(demand)*1000) for demand in model.state.demand[index]]
    data_model['num_vehicles'] = 100
    data_model['vehicle_capacities'] = [1000] * data_model['num_vehicles']
    data_model['depot'] = 0
    return data_model

# def create_data_model():
#     """Stores the data for the problem."""
#     data = {}
#     data['time_matrix'] = [
#         [0, 6, 9, 8, 7, 3, 6, 2, 3, 2, 6, 6, 4, 4, 5, 9, 7],
#         [6, 0, 8, 3, 2, 6, 8, 4, 8, 8, 13, 7, 5, 8, 12, 10, 14],
#         [9, 8, 0, 11, 10, 6, 3, 9, 5, 8, 4, 15, 14, 13, 9, 18, 9],
#         [8, 3, 11, 0, 1, 7, 10, 6, 10, 10, 14, 6, 7, 9, 14, 6, 16],
#         [7, 2, 10, 1, 0, 6, 9, 4, 8, 9, 13, 4, 6, 8, 12, 8, 14],
#         [3, 6, 6, 7, 6, 0, 2, 3, 2, 2, 7, 9, 7, 7, 6, 12, 8],
#         [6, 8, 3, 10, 9, 2, 0, 6, 2, 5, 4, 12, 10, 10, 6, 15, 5],
#         [2, 4, 9, 6, 4, 3, 6, 0, 4, 4, 8, 5, 4, 3, 7, 8, 10],
#         [3, 8, 5, 10, 8, 2, 2, 4, 0, 3, 4, 9, 8, 7, 3, 13, 6],
#         [2, 8, 8, 10, 9, 2, 5, 4, 3, 0, 4, 6, 5, 4, 3, 9, 5],
#         [6, 13, 4, 14, 13, 7, 4, 8, 4, 4, 0, 10, 9, 8, 4, 13, 4],
#         [6, 7, 15, 6, 4, 9, 12, 5, 9, 6, 10, 0, 1, 3, 7, 3, 10],
#         [4, 5, 14, 7, 6, 7, 10, 4, 8, 5, 9, 1, 0, 2, 6, 4, 8],
#         [4, 8, 13, 9, 8, 7, 10, 3, 7, 4, 8, 3, 2, 0, 4, 5, 6],
#         [5, 12, 9, 14, 12, 6, 6, 7, 3, 3, 4, 7, 6, 4, 0, 9, 2],
#         [9, 10, 18, 6, 8, 12, 15, 8, 13, 9, 13, 3, 4, 5, 9, 0, 9],
#         [7, 14, 9, 16, 14, 8, 5, 10, 6, 5, 4, 10, 8, 6, 2, 9, 0],
#     ]
#     data['time_windows'] = [
#         (0, 5),  # depot
#         (7, 12),  # 1
#         (10, 15),  # 2
#         (16, 18),  # 3
#         (10, 13),  # 4
#         (0, 5),  # 5
#         (5, 10),  # 6
#         (0, 4),  # 7
#         (5, 10),  # 8
#         (0, 3),  # 9
#         (10, 16),  # 10
#         (10, 15),  # 11
#         (0, 5),  # 12
#         (5, 10),  # 13
#         (7, 8),  # 14
#         (10, 15),  # 15
#         (11, 15),  # 16
#     ]
#     data['demands'] = [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
#     data['vehicle_capacities'] = [15, 15, 15, 15, 15, 15, 15, 15, 15, 15]

#     data['num_vehicles'] = 10
#     data['depot'] = 0
#     return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    time_dimension = routing.GetDimensionOrDie('Time')
    total_time = 0
    solution_list = []
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        temp = []
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            plan_output += '{0} Time({1},{2}) -> '.format(
                manager.IndexToNode(index),
                solution.Min(time_var),
                solution.Max(time_var)
            )
            temp.append(
                [manager.IndexToNode(index),
                 solution.Min(time_var)/1000,
                 solution.Max(time_var)/1000]
            )
            index = solution.Value(routing.NextVar(index))
        time_var = time_dimension.CumulVar(index)
        temp.append(
                [manager.IndexToNode(index),
                 solution.Min(time_var)/1000,
                 solution.Max(time_var)/1000]
            )
        if len(temp) != 1:
            solution_list.append(temp)
        total_time += solution.Min(time_var)
#     print('Total time of all routes: {}min'.format(total_time))
    return solution_list


def main(model, index):
    """Solve the VRP with time windows."""
    global data
    # Instantiate the data problem.
#     data = create_data_model()
    data = make_data_model(model, index)

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['time_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)
    
    # Create and register a transit callback.
    def time_callback(from_index, to_index):
        """Returns the travel time between the two nodes."""
        # Convert from routing variable Index to time matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['time_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(time_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Time Windows constraint.
    time = 'Time'
    routing.AddDimension(
        transit_callback_index,
        100000,  # allow waiting time
        100000,  # maximum time per vehicle
        False,  # Don't force start cumul to zero.
        time)
    time_dimension = routing.GetDimensionOrDie(time)
    
    # Add time window constraints for each location except depot.
    for location_idx, time_window in enumerate(data['time_windows']):
        if location_idx == data['depot']:
            continue
        index = manager.NodeToIndex(location_idx)
        time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])

    # Add time window constraints for each vehicle start node.
    depot_idx = data['depot']
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        time_dimension.CumulVar(index).SetRange(
            data['time_windows'][depot_idx][0],
            data['time_windows'][depot_idx][1])
        
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Instantiate route start and end times to produce feasible times.
    for i in range(data['num_vehicles']):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i)))
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.End(i)))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
#     search_parameters.local_search_metaheuristic = (
#         routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
#     search_parameters.time_limit.seconds = 30
#     search_parameters.log_search = True

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        total_time = print_solution(data, manager, routing, solution)
    return total_time


def ortools(model, index):
    return main(model, index)

In [16]:
totaltotal = 0
for i in range(100):
    solution_list = ortools(model, i)
    tot = 0
    for sol in solution_list:
        tot += sol[-1][2]
    print(tot)
    totaltotal += tot

105.176
108.94500000000001
129.033
108.53
99.113
108.565
106.81400000000001
111.289
103.241
115.39399999999999
96.76800000000001
126.332
117.725
111.37599999999999
124.24900000000001
125.837
112.655
115.44399999999999
104.736
108.365
112.77900000000001
109.983
105.526
122.262
126.12100000000001
104.184
107.851
111.271
121.319
116.51800000000003
118.298
104.067
122.71199999999999
112.40399999999998
113.236
110.86699999999999
107.989
107.77800000000002
120.014
115.116
117.83800000000001
117.76299999999998
104.08900000000001
114.81099999999999
122.13900000000002
105.55099999999999
129.249
114.12400000000001
113.36600000000001
98.96100000000001
111.931
109.993
120.594
114.249
110.459
103.023
132.777
106.97900000000001
101.412
108.815
97.40899999999999
131.066
101.15000000000002
103.66499999999999
132.673
97.769
92.71399999999998
106.78899999999999
113.87300000000002
115.56399999999998
113.32999999999998
109.49600000000001
109.64
133.586
103.76500000000001
106.92699999999999
111.53800000000

In [71]:
totaltotal

6987.743999999999

In [68]:
totaltotal

6959.5009999999975

In [18]:
solution_list

[[[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0,

In [ ]:
first step  adding time concept
second step refactor existing code
third step  modify endode layer (more practical one)
fourth step add time window as an instance
